In [ ]:
# Import standard Python modules.
import datetime
import os
import platform
import sys

# Import 3rd-party modules.
import matplotlib.pyplot as plt
import numpy as np

# Import TensorFlow.
import tensorflow as tf

: 

In [ ]:
# Use 64-bit math in TensorFlow.
tf.keras.backend.set_floatx("float64")

# Case 1: Incompressible, $\frac {dP} {dx}$ constant

Consider one-dimensional, incompressible fluid flow in a channel. For simplicity, assume the channel is a pipe with a circular cross-section of diameter $D$. Flow in the channel is controlled by the Bernoulli equation:

\begin{equation}
    P + \frac {1} {2} \rho u^2 + \rho g h = constant = C_1
\end{equation}

where $P$ is the pressure, $\rho$ is the mass density, $u$ is the fluid flow speed, $g$ is the acceleration due to gravity, and $h$ is the height of the fluid.

Neglecting gravity, this equation can be rearranged to solve for the flow speed $u$ as a function of the pressure $P$:

\begin{equation}
    u = \left[ \frac {2} {\rho} \left( C_1 - P \right) \right]^{\frac {1} {2}}
\end{equation}

Since the fluid is incompressible the density $\rho$ is constant.

The speed is therefore a function only of the pressure, as shown in the equation above.

The change in speed is computed using the chain rule:

\begin{equation}
    \frac {du} {dx} = \frac {du} {dP} \frac {dP} {dx} \\
    = \frac {1} {2} \left[ \frac {2} {\rho} \left( C_1 - P \right) \right]^{- \frac {1} {2}} \left( - \frac {2} {\rho} \frac {dP} {dx} \right) \\
    = - \left( \frac {1} {2 \rho} \right)^{ \frac {1} {2}} \left( C_1 - P \right)^{- \frac {1} {2}} \frac {dP} {dx}
\end{equation}

where $x$ is the linear distance along the channel.

Now assume the existence of a constant pressure gradient $\frac {dP} {dx}$:

\begin{equation}
    \frac {dP} {dx} = C_2 \Rightarrow P(x) = P_0 + C_2 x
\end{equation}

where $P_0$ is $P(x = 0)$.

Inserting the pressure gradient into the equation for the speed gradient, we get:

\begin{equation}
    \frac {du} {dx} = - \frac {C_2} { \left( 2 \rho \right)^{\frac {1} {2}}} \left( C_1 - P_0 - C_2 x \right)^{- \frac {1} {2}}
\end{equation}

Now assume $\rho = 1$, $P_0 = 1$, $u_0 = 1$, and $C_2 = -1$. The constant $C_1 = \frac {3} {2}$, and the differential equation becomes:

\begin{equation}
    \frac {du} {dx} = \frac {1} {\sqrt {2}} \left( x + \frac {1} {2} \right)^{- \frac {1} {2}} \\
    = \left( 2 x + 1 \right)^{- \frac {1} {2}}
\end{equation}

This ordinary differential equation has the analytical solution $u_a(x)$:

\begin{equation}
    u_a(x) = \sqrt {2} \left( x + \frac {1} {2} \right)^{\frac {1} {2}} \\
    = (2 x + 1)^{\frac {1} {2}}
\end{equation}

In [ ]:
eq_name = "case1"

# Define the analytical solution and derivative.
u_analytical = lambda x: (2*x + 1)**0.5
du_dx_analytical = lambda x: (2*x + 1)**-0.5

# Compute the analytical solution and derivative.
nx = 101
xa = np.linspace(0, 1, nx)
ua = u_analytical(xa)
dua_dx = du_dx_analytical(xa)

# Plot the analytical solution and derivative.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(xa, ua, label="$u_a$")
ax.plot(xa, dua_dx, label="$du_a/dx$")
ax.set_xlabel("x")
ax.set_ylabel("$u_a$ or $du_a/dx$")
ax.grid()
ax.legend()
ax.set_title("Analytical solution and derivative for %s" % eq_name)
plt.show()

# Solving the equation with a neural network

In [ ]:
def print_system_information():
    print("System report:")
    print(datetime.datetime.now())
    print("Host name: %s" % platform.node())
    print("OS: %s" % platform.platform())
    print("uname:", platform.uname())
    print("Python version: %s" % sys.version)
    print("Python build:", platform.python_build())
    print("Python compiler: %s" % platform.python_compiler())
    print("Python implementation: %s" % platform.python_implementation())
    # print("Python file: %s" % __file__)

In [ ]:
def create_output_directory(path=None):
    path_noext, ext = os.path.splitext(path)
    output_dir = path_noext
    if not os.path.exists(output_dir):
        os.mkdir(output_dir)
    return output_dir

In [ ]:
from nnde.math.trainingdata import create_training_grid2

def create_training_data(*n_train):
    x_train = np.linspace(0, 1, n_train[0])
    return x_train

In [ ]:
def build_model(H, w0_range, u0_range, v0_range):
    hidden_layer_1 = tf.keras.layers.Dense(
        units=H, use_bias=True,
        activation=tf.keras.activations.sigmoid,
        kernel_initializer=tf.keras.initializers.RandomUniform(*w0_range),
        bias_initializer=tf.keras.initializers.RandomUniform(*u0_range)
    )
    output_layer = tf.keras.layers.Dense(
        units=1,
        activation=tf.keras.activations.linear,
        kernel_initializer=tf.keras.initializers.RandomUniform(*v0_range),
        use_bias=False,
    )
    model = tf.keras.Sequential([hidden_layer_1, output_layer])
    return model

In [ ]:
print_system_information()

In [ ]:
# Set up the output directory.
path = os.path.join(".", eq_name)
output_dir = create_output_directory(path)

In [ ]:
# Define the hyperparameters.

# Training optimizer
optimizer_name = "Adam"

# Initial parameter ranges
w0_range = [-0.1, 0.1]
u0_range = [-0.1, 0.1]
v0_range = [-0.1, 0.1]

# Maximum number of training epochs.
max_epochs = 40000

# Learning rate.
learning_rate = 0.01

# Absolute tolerance for consecutive loss function values to indicate convergence.
tol = 1e-6

# Number of hidden nodes.
H = 10

# Number of dimensions
m = 1

# Number of training points in each dimension.
nx_train = 11
n_train = nx_train

# Random number generator seed.
random_seed = 0

In [ ]:
# Create and save the training data.
x_train = create_training_data(nx_train)
np.savetxt(os.path.join(output_dir, "x_train.dat"), x_train)

In [ ]:
# Define the differential equation using TensorFlow operations.

@tf.function
def ode_u(x, u, du_dx):
    G = du_dx - (2*x + 1)**-0.5
    return G

In [ ]:
# Define the trial function using TensorFlow operations.

@tf.function
def Y_trial_u(x, N):
    A = tf.constant([[1.0]], dtype="float64")
    P = x
    Y = A + P*N
    return Y

In [ ]:
# Build the model.
model_u = build_model(H, w0_range, u0_range, v0_range)

# Create the optimizer.
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Create history variables.
losses_u = []
losses = []
phist_u = []

# Set the random number seed for reproducibility.
tf.random.set_seed(random_seed)

# Rename the training data Variable for convenience, just for training.
# shape (n_train, m)
x_train_var = tf.Variable(np.array(x_train).reshape((n_train, m)), name="x_train")
x = x_train_var

# Clear the convergence flag to start.
converged = False

# Train the model.

print("Hyperparameters: n_train = %s, H = %s, max_epochs = %s, optimizer = %s, learning_rate = %s"
      % (n_train, H, max_epochs, optimizer_name, learning_rate))
t_start = datetime.datetime.now()
print("Training started at", t_start)

for epoch in range(max_epochs):

    # Run the forward pass.
    with tf.GradientTape(persistent=True) as tape1:
        with tf.GradientTape(persistent=True) as tape0:

            # Compute the network outputs at the training points.
            N_u = model_u(x)

            # Compute the trial solutions.
            u = Y_trial_u(x, N_u)

        # Compute the gradients of the trial solutions wrt inputs.
        du_dx = tape0.gradient(u, x)

        # Compute the estimates of the differential equations.
        G_u = ode_u(x, u, du_dx)

        # Compute the loss functions.
        L_u = tf.math.sqrt(tf.reduce_sum(G_u**2)/n_train)
        L = L_u

    # Save the current losses.
    losses_u.append(L_u)
    losses.append(L.numpy())

    # Check for convergence.
    if epoch > 0:
        loss_delta = losses[-1] - losses[-2]
        if abs(loss_delta) <= tol:
            converged = True
            break

    # Compute the gradient of the loss function wrt the network parameters.
    pgrad_u = tape1.gradient(L, model_u.trainable_variables)

    # Save the parameters used in this epoch.
    phist_u.append(
        np.hstack(
            (model_u.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
             model_u.trainable_variables[1].numpy(),       # u (H,) row vector
             model_u.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
        )
    )

    # Update the parameters for this epoch.
    optimizer.apply_gradients(zip(pgrad_u, model_u.trainable_variables))

    if epoch % 100 == 0:
        print("Ending epoch %s, loss function = %f" % (epoch, L.numpy()))

# Save the parameters used in the last epoch.
phist_u.append(
    np.hstack(
        (model_u.trainable_variables[0].numpy().reshape((m*H,)),    # w (m, H) matrix -> (m*H,) row vector
         model_u.trainable_variables[1].numpy(),       # u (H,) row vector
         model_u.trainable_variables[2][:, 0].numpy()) # v (H, 1) column vector
    )
)

n_epochs = epoch + 1

t_stop = datetime.datetime.now()
print("Training stopped at", t_stop)
t_elapsed = t_stop - t_start
print("Total training time was %s seconds." % t_elapsed.total_seconds())
print("Epochs: %d" % n_epochs)
print("Final value of loss function: %f" % losses[-1])
print("converged = %s" % converged)

# Save the parameter and loss function histories.
np.savetxt(os.path.join(output_dir, 'phist_u.dat'), np.array(phist_u))
np.savetxt(os.path.join(output_dir, 'losses.dat'), np.array(losses))

In [ ]:
# Compute and save the trained results at training points.
with tf.GradientTape(persistent=True) as tape:
    N_u = model_u(x)
    ut_train = Y_trial_u(x, N_u)
dut_dx_train = tape.gradient(ut_train, x)
np.savetxt(os.path.join(output_dir, "ut_train.dat"), ut_train.numpy().reshape((n_train,)))
np.savetxt(os.path.join(output_dir, "dut_dx_train.dat"), dut_dx_train.numpy())

# Compute and save the analytical solution and derivative at training points.
ua_train = u_analytical(x_train)
dua_dx_train = du_dx_analytical(x_train)
np.savetxt(os.path.join(output_dir,"ua_train.dat"), ua_train)
np.savetxt(os.path.join(output_dir,"dua_dx_train.dat"), dua_dx_train)

# Compute and save the error in the trained solution and derivative at training points.
ut_err_train = ut_train.numpy().reshape((nx_train,)) - ua_train
dut_dx_err_train = dut_dx_train.numpy().reshape((nx_train,)) - dua_dx_train
np.savetxt(os.path.join(output_dir, "ut_err_train.dat"), ut_err_train)
np.savetxt(os.path.join(output_dir, "dut_dx_err_train.dat"), dut_dx_err_train)

# Compute the final RMS error in the solution at the training points.
ut_rmse_train = np.sqrt(np.sum(ut_err_train**2)/n_train)
print("ut_rmse_train = %s" % ut_rmse_train)

In [ ]:
# Plot the loss function history.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.semilogy(losses)
ax.set_xlabel("Epoch")
ax.set_ylabel("Loss function (RMS error)")
ax.grid()
ax.set_title("Loss function evolution for %s" % eq_name)
plt.show()

In [ ]:
# Plot the the trained solution and derivative at the training points.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_train, ut_train, label="$u_t$")
ax.plot(x_train, dut_dx_train, label="$du_t/dx$")
ax.set_xlabel("x")
ax.set_ylabel("$u_t$ or $du_t/dx$")
ax.grid()
ax.legend()
ax.set_title("Trained solution and derivative for %s" % eq_name)
plt.show()

In [ ]:
# Plot the errors in the trained solution and derivative at the training points.
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(x_train, ut_err_train, label="$u_t - u_a$")
ax.plot(x_train, dut_dx_err_train, label="$du_t/dx - du_a/dx$")
ax.set_xlabel("x")
ax.set_ylabel("Trained - analytical")
ax.grid()
ax.legend()
ax.set_title("Error in trained solution for %s" % eq_name)
plt.show()